### 2.3.1 Enriching Word Vectors with Subword Information

#### 一. 介绍
1. 在此之前的embedding, 为单词表的每个单词赋予一个不同的向量. 单词向量之间没有参数共享, 也没有考虑过单词的形态. 这在形态意义丰富的语言中是个缺陷  
2. 此前的embedding只能给出训练集中出现过的单词的向量, 而这个文章可以根据单词的'subword'推测出训练集中不存在的单词的向量

#### 二. skip-gram负采样模型  
1. 给定一个长度为$W$的单词表, 目标是学习每个单词的向量表示.skip-gram采用最大化如下似然函数的方法$$\sum _{ t=1 }^{ T }{ \sum _{ c\in { C }_{ t } }^{  }{ \log { p\left( { w }_{ c }|{ w }_{ t } \right)  }  }  } $$其中, ${ w }_{ t }$为目标单词,${ w }_{ c }$为上下文  

2. 给定目标词,得到上下文词的概率可以假设为softmax概率, 即$$p\left( { w }_{ c }|{ w }_{ t } \right) =\frac { { e }^{ s\left( { w }_{ t },{ w }_{ c } \right)  } }{ \sum _{ j=1 }^{ W }{ { e }^{ s\left( { w }_{ t },{ w }_{ j } \right)  } }  } $$但是因为我们只想知道某一个上下文单词在给定目标单词后出现的概率是多少, 所以只要把问题相乘1个二分类问题. 从corpus中抽取目标词和上下文词作为正例, 目标词和随机选择单词表的词构成负例即可. 然后计算词向量.目标词和上下文的得分为 : $s\left( { w }_{ c }|{ w }_{ t } \right) ={ u }_{ { w }_{ c } }^{ T }*{ v }_{ { w }_{ t } }$

#### 三. subword模型
1. 使用子字表示单词  
 上述`skip-gram`没有考虑单词的形态, 这里我们将单词表示成一组'n个字符的集合'(bag of character n-gram).   
 为了区分单词的前后缀, 在单词开始和结尾分别加上符号'<'和'>'.因此, 单词'where'的3-gram表示为:   
 $<wh$ , $whe$ , $her$ , $ere$ , $re>$  
2. 单词的representation  
 假设我们得到了长度为$G$的`n-gram子字`(n个字符)词典.  
 假设一个单词w, 可以表示成子字集合${ g  }_{ w }=\left\{ 1,..,G \right\} $,集合中为单词的子字在字典中的index.  
  1. n的范围通常为[3,6]  
  2. 若每个子字g学习到一个向量$z_g$, 则单词$w$的向量表示为其子字集合${ g  }_{ w }$的和( sum ). 且两个单词w和c的得分函数 : $$s\left( w,c \right) =\sum _{ g\in { g  }_{ w } }^{  }{ { z }_{ g }^{ T }{ V }_{ c } } $$   
  3. 为了对子字单词表记忆, 使用hash函数, 将每一个子字转换成不同的integer数值.最终, 单词w被表示成其在word dictionary中的index和哈希后的子字数值集合
3. 使用子字模型训练词向量  
 同样采用上面的负采样二分类模型训练, 只是单词的表示变成了子字向量的和, 训练得到每个子字的表示

### 2.3.2 Bag of Tricks for Efficient Text Classification
本文讨论FastText用于标签预测和情感分类

#### 一. CBOW模型
1. CBOW和skip-gram是同时提出的向量化词的方法.都是基于2层神经网络的方法(1个隐藏层+1个分层softmax层)  
2. CBOW和skip-gram的区别在于  
  1. CBOW的输入层为target周围的n个context单词, 输出为target单词
  2. softmax输入为target单词, 输出为target单词周围n个位置, 有该context单词的概率
  
<img src='img/cbowandskipgram.png' height='60%' width='60%'>
3. CBOW和skip-gram是基于2层神经网络训练出来的, 而带负采样的skip-gram才是逻辑回归训练出来的
4. 比较各种方法训练出来的词向量好坏的一种方法是:   
 给定2组词, 看其相似度. 比如"man-women"相似于'king-queen'
 



#### 二. FastText模型
1. fasttext用于文本分类问题. fasttext得到的词向量是文本分类的副产物.
2. fasttext是有监督模型,和CBOW网络结构类似, 通过context预测标签. 与其区别主要在2个地方的优化:  
  1. fasttext直接进行文本分类, 因此fasttext的输出标签是文本类别, 而CBOW输出的是target word  
  2. fasttext使用分层softmax和n-gram优化训练效果  
3. 分层softmax:   
 1. 若使用传统softmax计算文本分类时, 当类别很多时计算量相当大. 时间复杂度为$O(kh)$. k为类别个数, h为文本向量长度  
 2. 若使用霍夫曼树构成的分层softmax, 则时间复杂度变为$O(hlog_2(k))$
 3. 分层softmax在测试阶段, 输出所属类别的概率时在速度上也很有优势.若节点深度为$l+1$,父节点为$\left( { n }_{ 1 }...{ n }_{ l } \right) $  
 则输出其类别概率为$$p(n_{l+1})=\prod _{ i=1 }^{ l }{ p\left( { n }_{ i } \right)  } $$  
4. 使用句子的n-gram表示局部信息  
 因为输入的训练数据是上下文的单词, 但这种输入彼此之间独立, 没有考虑词的顺序带来的语义问题.因此fastext在输入端使用n-gram模型把语句的局部顺序作为新特征一起训练. 具体的:  
   "I eat apple"的2元模型为: "I eat"和"eat apple"

In [9]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division
import pandas as pd
import numpy as np
import jieba
import subprocess,os

In [5]:
# 1.加载数据文件
neg_df = pd.read_excel('/home/lj/Downloads/neg.xls',header=None,index=None)
pos_df = pd.read_excel('/home/lj/Downloads/pos.xls',header=None,index=None)

pos_corpus = pos_df[0]
neg_corpus = neg_df[0]
pos_label = 1
neg_label = 0

pos_corpus.head()

0    做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1    作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2    作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3    作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4    作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...
Name: 0, dtype: object

#### 整理文件格式为: content \t__label__{classname}

In [6]:
# 对每个语句分词
def cutword(_sentence,stp_list):
    semiangle_str = [strQ2B(w) for w in jieba.cut(_sentence)]
    cut_words = [w for w in semiangle_str if w not in stp_list]
    return ' '.join(cut_words)

def strQ2B(ustring):
    ''' ustring : 需要转换成半角的字符串 '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss

In [9]:
stp_list = open('../../data/stop_words_utf8.txt').read().splitlines()

# 将corpus写成fasttext可解析的结构
new_stenence = ''
for sentence in pos_corpus:
    new_stenence += "%s%s%s\n"%(cutword(sentence,stp_list),'\t__label__',1)
for sentence in neg_corpus:
    new_stenence += "%s%s%s\n"%(cutword(sentence,stp_list),'\t__label__',0)
with open('../../data/fasttext_corpus','w') as f:
    f.write(new_stenence)

#### 执行fasttext命令
```bash
fasttext supervised -input ../../data/fasttext_corpus -output fasttext_model  
```  
输出两个文件 : '模型文件'和'中间得到的word embedding'

In [11]:
def execute_command(command):
    p = subprocess.Popen(command,
                         cwd=os.path.abspath('.'),
                         stdout = subprocess.PIPE,
                         stdin = subprocess.PIPE,
                         stderr=subprocess.PIPE,
                         shell=True)
    stdout, stderr = p.stdout,p.stderr
    print('以下是执行结果:')
    print(stdout.read().decode('utf-8'))
    print(stderr.read().decode('utf-8'))
    
input_path = '../../data/fasttext_corpus'
output_path = 'fasttext_model'
command = 'fasttext supervised -input %s -output %s' % (input_path,output_path)
print(command)
execute_command(command)

fasttext supervised -input ../../data/fasttext_corpus -output fasttext_model
以下是执行结果:

Read 0M words
Number of words:  49954
Number of labels: 2
Progress: 100.0% words/sec/thread: 1451252 lr:  0.000000 loss:  0.211923 ETA:   0h 0m



#### fasttext预测

In [26]:
def handle_test_str(sentences):
    with open('test_fasttext.txt','w') as f:
        content = ''
        for sentence in sentences:
            content += '%s\n' % (' '.join(jieba.lcut(sentence)))
        f.write(content)
        
# 将待测试的评论分词后写入文件, 以供后续fasttext测试
handle_test_str(['真傻逼','我很喜欢'])

test_command = 'fasttext predict %s.bin %s ' % (output_path,'test_fasttext.txt')
print(test_command)
execute_command(test_command)

fasttext predict fasttext_model.bin test_fasttext.txt 
以下是执行结果:
__label__0
__label__1


